In [798]:
from bs4 import BeautifulSoup as bs
import requests, json
from datetime import datetime

In [799]:

def get_usd_curreny():
    dt = datetime.today()
    today = dt.strftime('%Y%m%d')

    data = {"root_info":{"serviceType":"GU","serviceCode":"F3730","nextServiceCode":"","pkcs7Data":"","signCode":"","signData":"","useSign":"","useCert":"","permitMultiTransaction":"","keepTransactionSession":"","skipErrorMsg":"","mode":"","language":"ko","exe2e":"","hideProcess":"","clearTarget":"data:json,[{\"id\":\"dm_R_RIBF3730\",\"key\":\"R_RIBF3730\"},{\"id\":\"dl_R_RIBF3730_1\",\"key\":\"R_RIBF3730_1\"},{\"id\":\"dl_R_RIBF3730_2\",\"key\":\"R_RIBF3730_2\"}]","callBack":"shbObj.fncF3730Callback","exceptionCallback":"","requestMessage":"","responseMessage":"","serviceOption":"","pcLog":"","preInqForMulti":"","makesum":"","removeIndex":"","redirectUrl":"","preInqKey":"","_multi_transfer_":"","_multi_transfer_count_":"","_multi_transfer_amt_":"","userCallback":"","menuCode":"","certtype":""},
            "S_RIBF3730":{"조회구분":"","조회일자":today,"고시회차":"1","조회일자_display":""}}
    url = 'https://bank.shinhan.com/comjsp/dataProcess.jsp?svr_type=GU&svr_code=F3730'
    headers = {'Content-Tpe': 'application/json; charset=utf-8'}
    res = requests.post(url, data=json.dumps(data))


    currency_exchanges = res.json()['R_RIBF3730_1']
    return {'code': currency_exchanges[0]['통화CODE'], 'cur': currency_exchanges[0]['전신환매도환율']}
    #전신환매도환율
    #통화CODE
usd_cur = get_usd_curreny()

In [800]:
usd_cur['cur']

'1296.40'

In [801]:
import math

def round_off(num, pos=2):
    t = str(num).split('.')
    t[1] = t[1][:pos]
    return float(".".join(t))

In [810]:
usd_cur = {'code': 'USD', 'cur': '1296.40'}

card_fee = 0

def exCalc(v):
    visafee = v * 0.011;
    visafee = float(round_off(visafee));
    t = round(v * float(usd_cur['cur']))
    card_fee = t * 0.0018
    card_fee = round_off(card_fee, 0)
    krw = ((v + visafee) * float(usd_cur['cur'])) + card_fee
    return (v+visafee, int(krw), card_fee)

exCalc(6.81)

(6.88, 8934, 15.0)

In [811]:
def default_value():
    tar = 5999
    for i in range(600, 400, -1):
        usd = i/100
        usdv, krw, card_fee = exCalc(usd)
        if (krw < 6000):
            return (usd, krw, card_fee, usdv)
        

usd, krw, card_fee, usdv = default_value()
usd, krw, card_fee, usdv

(4.57, 5999, 10.0, 4.62)

In [812]:
currenys = []
def get_currency_rate():
    currencys_list = ["USD", "AED", "ARS", "AUD", "CAD", "CHF", "CLP", "CZK", "CNY", "DKK", "EUR", "GBP", "HKD", "HUF", "IDR", "INR", "JPY", "NPR", "NGN", "NZD", "NOK", "MXN", "MYR", "PHP", "PLN", "RUB", "SAR", "SGD", "SEK", "THB", "TRY", "TWD", "VND", "KRW"]

    currenys = []
    dt = datetime.today()
    today = dt.strftime('/%m/%d/%Y')

    for cur in currencys_list:
        url = 'https://www.visakorea.com/travel-with-visa/exchange-rate-calculator.html?fromCurr=USD&toCurr={}&fee=0&utcConvertedDate={}&exchangedate={}&submitButton=Calculate+Exchange+Rates'.format(cur, today, today)
        page = requests.get(url)
        soup = bs(page.text, "html.parser")
        currency_rate = soup.select('#er_result_table > tr > td > p:nth-child(2) > span > strong:nth-child(2)')[0].get_text()
        currenys.append({cur: {'rate': usd_cur['cur'], 'currency_rate': currency_rate if currency_rate != '' else '1.0'}})
    return currenys


In [813]:
currenys = get_currency_rate()

In [814]:
currenys

[{'USD': {'rate': '1296.40', 'currency_rate': '1.0'}},
 {'AED': {'rate': '1296.40', 'currency_rate': '0.2723086349'}},
 {'ARS': {'rate': '1296.40', 'currency_rate': '0.008142361'}},
 {'AUD': {'rate': '1296.40', 'currency_rate': '0.705599294'}},
 {'CAD': {'rate': '1296.40', 'currency_rate': '0.773035714'}},
 {'CHF': {'rate': '1296.40', 'currency_rate': '1.0398242692'}},
 {'CLP': {'rate': '1296.40', 'currency_rate': '0.0011427265'}},
 {'CZK': {'rate': '1296.40', 'currency_rate': '0.0427404795'}},
 {'CNY': {'rate': '1296.40', 'currency_rate': '0.1491908513'}},
 {'DKK': {'rate': '1296.40', 'currency_rate': '0.1420009372'}},
 {'EUR': {'rate': '1296.40', 'currency_rate': '1.056198944'}},
 {'GBP': {'rate': '1296.40', 'currency_rate': '1.238998761'}},
 {'HKD': {'rate': '1296.40', 'currency_rate': '0.1274046375'}},
 {'HUF': {'rate': '1296.40', 'currency_rate': '0.0026549116'}},
 {'IDR': {'rate': '1296.40', 'currency_rate': '0.0000676041'}},
 {'INR': {'rate': '1296.40', 'currency_rate': '0.01283

In [818]:
def get_info_with_currency_and_value(cur, v):
    idx = currencys_list.index(cur)
    return exCalc(round(v * float(currenys[idx][cur]['currency_rate']), 2))

In [819]:
exCalc(4.34)

(4.38, 5688, 10.0)

In [846]:
a = get_info_with_currency_and_value('AED', 25)
a

(6.88, 8934, 15.0)

In [824]:
def get_all_info_currency_with_value():
    currencys_list = ["USD", "AED", "ARS", "AUD", "CAD", "CHF", "CLP", "CZK", "CNY", "DKK", "EUR", "GBP", "HKD", "HUF", "IDR", "INR", "JPY", "NPR", "NGN", "NZD", "NOK", "MXN", "MYR", "PHP", "PLN", "RUB", "SAR", "SGD", "SEK", "THB", "TRY", "TWD", "VND", "KRW"]
    comma = [0,2,0,2,2,2,0,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0]
    usd, krw, card_fee, usdv = default_value()

    for idx, cur in enumerate(currencys_list):
        for i in range(10, 20000000, 1):
            t = i/100
            a = t * float(currenys[idx][cur]['currency_rate']) 
            amount = 0
            if (float("{}{}".format(str(usd), 5)) <= a):                
                prev = t - 0.01
                if (comma[idx] == 0):
                    amount = round_off(prev, 0) 
                else:
                    amount = round(prev, 2)
                currenys[idx][cur]['amount'] = amount
                currenys[idx][cur]['usd'] = usdv
                currenys[idx][cur]['krw'] = krw
                currenys[idx][cur]['card_fees'] = card_fee
                break
    return currenys

In [825]:
get_all_info_currency_with_value()

[{'USD': {'rate': '1296.40',
   'currency_rate': '1.0',
   'amount': 4.0,
   'usd': 4.62,
   'krw': 5999,
   'card_fees': 10.0}},
 {'AED': {'rate': '1296.40',
   'currency_rate': '0.2723086349',
   'amount': 16.8,
   'usd': 4.62,
   'krw': 5999,
   'card_fees': 10.0}},
 {'ARS': {'rate': '1296.40',
   'currency_rate': '0.008142361',
   'amount': 561.0,
   'usd': 4.62,
   'krw': 5999,
   'card_fees': 10.0}},
 {'AUD': {'rate': '1296.40',
   'currency_rate': '0.705599294',
   'amount': 6.48,
   'usd': 4.62,
   'krw': 5999,
   'card_fees': 10.0}},
 {'CAD': {'rate': '1296.40',
   'currency_rate': '0.773035714',
   'amount': 5.91,
   'usd': 4.62,
   'krw': 5999,
   'card_fees': 10.0}},
 {'CHF': {'rate': '1296.40',
   'currency_rate': '1.0398242692',
   'amount': 4.39,
   'usd': 4.62,
   'krw': 5999,
   'card_fees': 10.0}},
 {'CLP': {'rate': '1296.40',
   'currency_rate': '0.0011427265',
   'amount': 4003.0,
   'usd': 4.62,
   'krw': 5999,
   'card_fees': 10.0}},
 {'CZK': {'rate': '1296.40',
 

In [826]:
currenys

[{'USD': {'rate': '1296.40',
   'currency_rate': '1.0',
   'amount': 4.0,
   'usd': 4.62,
   'krw': 5999,
   'card_fees': 10.0}},
 {'AED': {'rate': '1296.40',
   'currency_rate': '0.2723086349',
   'amount': 16.8,
   'usd': 4.62,
   'krw': 5999,
   'card_fees': 10.0}},
 {'ARS': {'rate': '1296.40',
   'currency_rate': '0.008142361',
   'amount': 561.0,
   'usd': 4.62,
   'krw': 5999,
   'card_fees': 10.0}},
 {'AUD': {'rate': '1296.40',
   'currency_rate': '0.705599294',
   'amount': 6.48,
   'usd': 4.62,
   'krw': 5999,
   'card_fees': 10.0}},
 {'CAD': {'rate': '1296.40',
   'currency_rate': '0.773035714',
   'amount': 5.91,
   'usd': 4.62,
   'krw': 5999,
   'card_fees': 10.0}},
 {'CHF': {'rate': '1296.40',
   'currency_rate': '1.0398242692',
   'amount': 4.39,
   'usd': 4.62,
   'krw': 5999,
   'card_fees': 10.0}},
 {'CLP': {'rate': '1296.40',
   'currency_rate': '0.0011427265',
   'amount': 4003.0,
   'usd': 4.62,
   'krw': 5999,
   'card_fees': 10.0}},
 {'CZK': {'rate': '1296.40',
 

In [828]:
cur

['USD',
 'THB',
 'SGD',
 'MYR',
 'CNY',
 'IDR',
 'AUD',
 'EUR',
 'HKD',
 'INR',
 'VND',
 'PHP',
 'GBP',
 'JPY',
 'TWD']

In [882]:
import re
cur = ["USD", "THB", "SGD", "MYR", "CNY", "IDR", "AUD", "EUR", "HKD", "INR", "VND", "PHP", "GBP", "JPY", "TWD"]

url = 'https://www.qoo10.com/item/Q-COIN-TOP-UP-VOUCHER-BY-QOO10-SG/705045258?banner_no=185296'
qcoin_list = []
for c in cur:
    headers = { "cookie": "gmktCurrency={}".format(c)}

    page = requests.get(url, headers=headers)

    soup = bs(page.text, "html.parser")
    price = soup.select('#qprice_span')[0]['data-price']
    

    for i in range(32, 40, 1):
        _, krw, _ = get_info_with_currency_and_value(c, i * float(price))
        if (krw > 6000):
            count = i - 1
            amount = (i - 1) * float(price)
            _, krw, _ = get_info_with_currency_and_value(c, amount)
            rank = count + (krw / 59990)
            print((krw / 59990))
            qcoin_list.append({"cur": c, "price": price, "count": count, "krw": krw, 'amount': amount, "rank": rank})
            break
qcoin_list = sorted(qcoin_list, key=lambda d: d['rank'], reverse=True) 

        




0.0997832972162027
0.0997832972162027
0.0995665944324054
0.0995665944324054
0.0989164860810135
0.0982663777296216
0.1
0.0997832972162027
0.09783297216202701
0.0973995665944324
0.0973995665944324
0.0997832972162027
0.0973995665944324
0.0982663777296216
0.0997832972162027


In [883]:
qcoin_list

[{'cur': 'USD',
  'price': '0.12',
  'count': 38,
  'krw': 5986,
  'amount': 4.56,
  'rank': 38.099783297216206},
 {'cur': 'THB',
  'price': '4.19',
  'count': 38,
  'krw': 5986,
  'amount': 159.22000000000003,
  'rank': 38.099783297216206},
 {'cur': 'SGD',
  'price': '0.17',
  'count': 37,
  'krw': 5973,
  'amount': 6.29,
  'rank': 37.0995665944324},
 {'cur': 'MYR',
  'price': '0.54',
  'count': 37,
  'krw': 5973,
  'amount': 19.98,
  'rank': 37.0995665944324},
 {'cur': 'CNY',
  'price': '0.82',
  'count': 37,
  'krw': 5934,
  'amount': 30.34,
  'rank': 37.098916486081016},
 {'cur': 'IDR',
  'price': '1800',
  'count': 37,
  'krw': 5895,
  'amount': 66600.0,
  'rank': 37.09826637772962},
 {'cur': 'HKD',
  'price': '0.95',
  'count': 37,
  'krw': 5869,
  'amount': 35.15,
  'rank': 37.097832972162024},
 {'cur': 'INR',
  'price': '9.39',
  'count': 37,
  'krw': 5843,
  'amount': 347.43,
  'rank': 37.09739956659443},
 {'cur': 'VND',
  'price': '2800',
  'count': 37,
  'krw': 5843,
  'amou

In [869]:
5936/5999

0.9894982497082847